In [ ]:
import pandas as pd
import numpy as np
from typing import List, Dict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import regex as re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download ('stopwords')
from nltk.corpus import stopwords 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/dataset/IRproject

Mounted at /content/drive
/content/drive/My Drive/dataset/IRproject


In [ ]:
songs = pd.read_csv('songdata.csv')
songs

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...


In [ ]:
songs2 = songs
songs = songs2[0:10000]

In [ ]:
songs

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
9995,Justin Bieber,All Bad,/j/justin+bieber/all+bad_21071754.html,"[Verse:] \nIt's another, if it ain't one thin..."
9996,Justin Bieber,All I Want Is You,/j/justin+bieber/all+i+want+is+you_20985618.html,Sitting here all alone watching the snow fall ...
9997,Justin Bieber,As Long As You Love Me,/j/justin+bieber/as+long+as+you+love+me_210218...,"As long as you love me (love me, love me) \nA..."
9998,Justin Bieber,Baby Lady,/j/justin+bieber/baby+lady_20467491.html,Put me in a outfit from the gap like that. One...


In [ ]:
def preprocess(texts):
  stopword = stopwords.words('english')
  clean_text = ' '.join([word for word in texts.split()if word not in stopword]) 
  clean_text = re.sub('\n','',clean_text)
  clean_text = clean_text.lower()
  clean_text = re.sub(r'\[[^)]*\]','',clean_text)
  return clean_text


In [ ]:
songs['Cleaned_text'] = songs['text'].apply(lambda text: preprocess(text))
songs

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,artist,song,link,text,Cleaned_text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...","look face, wonderful face and means something ..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...","take easy me, please touch gently like summer ..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,i'll never know i go why i put lousy rotten sh...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,making somebody happy question give take you l...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,making somebody happy question give take you l...
...,...,...,...,...,...
9995,Justin Bieber,All Bad,/j/justin+bieber/all+bad_21071754.html,"[Verse:] \nIt's another, if it ain't one thin...","it's another, ain't one thing instigators, li..."
9996,Justin Bieber,All I Want Is You,/j/justin+bieber/all+i+want+is+you_20985618.html,Sitting here all alone watching the snow fall ...,sitting alone watching snow fall looking back ...
9997,Justin Bieber,As Long As You Love Me,/j/justin+bieber/as+long+as+you+love+me_210218...,"As long as you love me (love me, love me) \nA...","as long love (love me, love me) as long love (..."
9998,Justin Bieber,Baby Lady,/j/justin+bieber/baby+lady_20467491.html,Put me in a outfit from the gap like that. One...,put outfit gap like that. one matches with jac...


In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
lyrics_matrix = tfidf.fit_transform(songs['Cleaned_text'])

In [ ]:
cosine_similarities = cosine_similarity(lyrics_matrix)

In [ ]:

similarities = {}
for i in range(len(cosine_similarities)):
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices]

In [ ]:
def _print(song, recom_song):
    item = len(recom_song)
        
    print('Recommended songs for {song} are:')
    for i in range(item):
      print(i+1)
      print(f"{recom_song[i][1]} by {recom_song[i][2]}")
      print(f"Similarity score : {round(recom_song[i][0], 3)}")
        
def recommend(song,number_songs,matrix_similar):
    recom_song_temp = matrix_similar[song][:(number_songs+1)]
    recom_song = recom_song_temp[1:number_songs+1]
    _print(song=song, recom_song=recom_song)


In [ ]:
selected_song = songs['song'].iloc[119]
number_song = input("Enter number of recommendations you want to receive : ")
number_song_int = int(number_song)

Enter number of recommendations you want to receive : 5


In [ ]:
recommend(selected_song,number_song_int, similarities)

Recommended songs for {song} are:
1
Hey Hey by Eric Clapton
Similarity score : 0.427
2
The Best Damn Thing by Avril Lavigne
Similarity score : 0.4
3
Lay It All Down by John Martyn
Similarity score : 0.4
4
Weird World by Backstreet Boys
Similarity score : 0.378
5
Top Down by Fifth Harmony
Similarity score : 0.364
